In [1]:
%load_ext autoreload
%autoreload 1

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [2]:
import ControlBurn as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix as cm
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.linear_model import LogisticRegression as lr
from sklearn.linear_model import RidgeClassifier as rc
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA

In [3]:
df_rain = pd.read_csv('datasets/weatherAUS_clean.csv')
rain_feats = df_rain.columns.values
rain_x = df_rain[rain_feats[1:-1]]
rain_y = df_rain[rain_feats[-1]]
rain_x_tr, rain_x_te, rain_y_tr, rain_y_te = train_test_split(rain_x, rain_y, test_size=0.2)

In [4]:
# rain_norm_x = rain_x - rain_x.min()
# rain_norm_x /= (rain_norm_x.max() - rain_norm_x.min())
rain_norm_x = rain_x - rain_x.mean()
rain_norm_x_tr, rain_norm_x_te, rain_norm_y_tr, rain_norm_y_te = train_test_split(rain_norm_x, rain_y, test_size=0.2)

In [5]:
df_rain['RainTomorrow'].max()

1.0

In [6]:
rain_x.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindGustDir_E,WindGustDir_W,WindDir9am_N,WindDir9am_S,WindDir9am_E,WindDir9am_W,WindDir3pm_N,WindDir3pm_S,WindDir3pm_E,WindDir3pm_W
0,13.4,22.9,0.6,24.666201,7.20069,44.0,20.0,24.0,71.0,22.0,...,0.0,6.0,0.0,0.0,0.0,6.0,2.0,0.0,0.0,4.0
1,7.4,25.1,0.0,24.666201,7.20069,44.0,4.0,22.0,44.0,25.0,...,0.0,4.0,4.0,0.0,0.0,2.0,0.0,2.0,0.0,4.0
2,12.9,25.7,0.0,24.666201,7.20069,46.0,19.0,26.0,38.0,30.0,...,0.0,4.0,0.0,0.0,0.0,6.0,0.0,2.0,0.0,4.0
3,9.2,28.0,0.0,24.666201,7.20069,24.0,11.0,9.0,45.0,16.0,...,3.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,6.0,0.0
4,17.5,32.3,1.0,24.666201,7.20069,41.0,7.0,20.0,82.0,33.0,...,0.0,6.0,2.0,0.0,4.0,0.0,3.0,0.0,0.0,3.0


In [7]:
rain_norm_x.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindGustDir_E,WindGustDir_W,WindDir9am_N,WindDir9am_S,WindDir9am_E,WindDir9am_W,WindDir3pm_N,WindDir3pm_S,WindDir3pm_E,WindDir3pm_W
0,1.007883,-0.542506,-1.756095,11.25839,-0.287912,3.313458,4.970112,4.79261,3.314295,-28.79899,...,-1.469488,4.437154,-1.580598,-1.514377,-1.522596,4.617571,0.618491,-1.609254,-1.443485,2.434247
1,-4.992117,1.657494,-2.356095,11.25839,-0.287912,3.313458,-11.029888,2.79261,-23.685705,-25.79899,...,-1.469488,2.437154,2.419402,-1.514377,-1.522596,0.617571,-1.381509,0.390746,-1.443485,2.434247
2,0.507883,2.257494,-2.356095,11.25839,-0.287912,5.313458,3.970112,6.79261,-29.685705,-20.79899,...,-1.469488,2.437154,-1.580598,-1.514377,-1.522596,4.617571,-1.381509,0.390746,-1.443485,2.434247
3,-3.192117,4.557494,-2.356095,11.25839,-0.287912,-16.686542,-4.029888,-10.20739,-22.685705,-34.79899,...,1.530512,-1.562846,-1.580598,1.485623,1.477404,-1.382429,-1.381509,-1.609254,4.556515,-1.565753
4,5.107883,8.857494,-1.356095,11.25839,-0.287912,0.313458,-8.029888,0.79261,14.314295,-17.79899,...,-1.469488,4.437154,0.419402,-1.514377,2.477404,-1.382429,1.618491,-1.609254,-1.443485,1.434247


In [8]:
rain_norm_x.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindGustDir_E,WindGustDir_W,WindDir9am_N,WindDir9am_S,WindDir9am_E,WindDir9am_W,WindDir3pm_N,WindDir3pm_S,WindDir3pm_E,WindDir3pm_W
count,1.217900e+05,1.217900e+05,1.217900e+05,1.217900e+05,1.217900e+05,1.217900e+05,1.217900e+05,1.217900e+05,1.217900e+05,1.217900e+05,...,1.217900e+05,1.217900e+05,1.217900e+05,1.217900e+05,1.217900e+05,1.217900e+05,1.217900e+05,1.217900e+05,1.217900e+05,1.217900e+05
mean,1.400349e-13,2.120835e-13,-2.429887e-13,1.102549e-11,7.201489e-12,-1.004410e-15,-3.285801e-16,-9.035952e-16,3.584510e-16,-3.942961e-15,...,-1.941610e-16,-2.389673e-16,-1.120159e-16,-1.484211e-16,-1.344191e-16,-1.642900e-16,-5.227410e-17,1.344191e-16,1.530884e-16,6.720956e-17
std,6.336322e+00,7.140863e+00,8.482433e+00,9.921268e+00,2.807379e+00,1.339052e+01,8.321005e+00,8.587120e+00,1.905808e+01,2.093924e+01,...,1.921960e+00,1.966491e+00,2.016462e+00,1.929320e+00,1.924537e+00,1.872716e+00,1.882703e+00,1.942573e+00,1.882713e+00,1.961487e+00
min,-2.089212e+01,-2.824251e+01,-2.356095e+00,-1.340781e+01,-7.488602e+00,-3.368654e+01,-1.302989e+01,-1.720739e+01,-6.768570e+01,-5.079899e+01,...,-1.469488e+00,-1.562846e+00,-1.580598e+00,-1.514377e+00,-1.522596e+00,-1.382429e+00,-1.381509e+00,-1.609254e+00,-1.443485e+00,-1.565753e+00
25%,-4.592117e+00,-5.342506e+00,-2.356095e+00,-9.407811e+00,-2.879119e-01,-9.686542e+00,-6.029888e+00,-6.207390e+00,-1.168570e+01,-1.479899e+01,...,-1.469488e+00,-1.562846e+00,-1.580598e+00,-1.514377e+00,-1.522596e+00,-1.382429e+00,-1.381509e+00,-1.609254e+00,-1.443485e+00,-1.565753e+00
50%,-1.921168e-01,-4.425060e-01,-2.356095e+00,-4.407811e+00,-2.879119e-01,-1.686542e+00,-2.029888e+00,-2.073898e-01,1.314295e+00,2.010099e-01,...,-1.469488e+00,-1.562846e+00,-1.580598e+00,-1.514377e+00,-1.522596e+00,-1.382429e+00,-1.381509e+00,-1.609254e+00,-1.443485e+00,-1.565753e+00
75%,4.607883e+00,5.057494e+00,-1.556095e+00,1.125839e+01,1.611398e+00,7.313458e+00,4.970112e+00,4.792610e+00,1.431430e+01,1.420101e+01,...,1.530512e+00,1.437154e+00,1.419402e+00,1.485623e+00,1.477404e+00,1.617571e+00,1.618491e+00,1.390746e+00,1.556515e+00,1.434247e+00
max,2.150788e+01,2.465749e+01,3.652439e+02,6.899219e+01,7.011398e+00,9.431346e+01,7.197011e+01,6.779261e+01,3.231430e+01,4.920101e+01,...,4.530512e+00,4.437154e+00,4.419402e+00,4.485623e+00,4.477404e+00,4.617571e+00,4.618491e+00,4.390746e+00,4.556515e+00,4.434247e+00


In [9]:
rain_feats

array(['Unnamed: 0', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm',
       'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm',
       'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm', 'RainToday',
       'WindGustDir_N', 'WindGustDir_S', 'WindGustDir_E', 'WindGustDir_W',
       'WindDir9am_N', 'WindDir9am_S', 'WindDir9am_E', 'WindDir9am_W',
       'WindDir3pm_N', 'WindDir3pm_S', 'WindDir3pm_E', 'WindDir3pm_W',
       'RainTomorrow'], dtype=object)

In [10]:
rain_y_te_true = rain_y_te[rain_y_te == 1]
rain_x_te_true = rain_x_te[rain_y_te == 1]
rain_y_te_false = rain_y_te[rain_y_te == 0]
rain_x_te_false = rain_x_te[rain_y_te == 0]
rain_norm_y_te_true = rain_norm_y_te[rain_norm_y_te == 1]
rain_norm_x_te_true = rain_norm_x_te[rain_norm_y_te == 1]
rain_norm_y_te_false = rain_norm_y_te[rain_norm_y_te == 0]
rain_norm_x_te_false = rain_norm_x_te[rain_norm_y_te == 0]

In [ ]:
U, S, V = np.linalg.svd(rain_x_tr[:5000])
Un, Sn, Vn = np.linalg.svd(rain_norm_x_tr[:5000])

In [ ]:
print(S)
plt.plot(S)
# demonstrate features selected by PCA would do poorly in RF but do well with lasso
# normalize every feature and try again

In [ ]:
Sn
plt.plot(Sn)

In [ ]:
clf = lr(penalty='l2').fit(rain_x_tr, rain_y_tr)
print(clf.score(rain_x_te, rain_y_te), clf.score(rain_x_te_true, rain_y_te_true), clf.score(rain_x_te_false, rain_y_te_false))
print('Coefficients:', clf.coef_)

In [ ]:
clf = lr(penalty='l1', solver='liblinear').fit(rain_x_tr, rain_y_tr)
print(clf.score(rain_x_te, rain_y_te), clf.score(rain_x_te_true, rain_y_te_true), clf.score(rain_x_te_false, rain_y_te_false))
print('Coefficients:', clf.coef_)

In [ ]:
# run pca for different numbers of output features, see how loss changes
# comparison with lasso (ridge)
# and RF to see loss progression
rain_norm_x_tr.describe()

In [ ]:
cmatrices = []
cmatricesn = []
scores = []
scoresn = []
for i in range(1, 30):
    pca = PCA(n_components=i)
    pcan = PCA(n_components=i)
    pca.fit(rain_x_tr)
    pcan.fit(rain_norm_x_tr)
    
    clf = lr(penalty='l2').fit(pca.transform(rain_x_tr), rain_y_tr)
    clfn = lr(penalty='l2').fit(pcan.transform(rain_norm_x_tr), rain_y_tr)
    
    cmatrices.append(cm(rain_y_te, clf.predict(pca.transform(rain_x_te))))
    cmatricesn.append(cm(rain_norm_y_te, clfn.predict(pcan.transform(rain_norm_x_te))))
    
    scores.append(clf.score(pca.transform(rain_x_te_true), rain_y_te_true))
    scoresn.append(clfn.score(pcan.transform(rain_norm_x_te_true), rain_norm_y_te_true))

In [ ]:
plt.plot(scores)

In [ ]:
plt.plot(scoresn)

In [ ]:
cmatricesn

In [13]:
cmatrices = []
cmatricesn = []
scores = []
scoresn = []
for i in range(1, 30):
    pca = KernelPCA(n_components=i, kernel='poly')
    pcan = KernelPCA(n_components=i, kernel='poly')
    pca.fit(rain_x_tr[:5000])
    pcan.fit(rain_norm_x_tr[:5000])
    
    clf = lr(penalty='l2').fit(pca.transform(rain_x_tr), rain_y_tr)
    clfn = lr(penalty='l2').fit(pcan.transform(rain_norm_x_tr), rain_y_tr)
    
    cmatrices.append(cm(rain_y_te, clf.predict(pca.transform(rain_x_te))))
    cmatricesn.append(cm(rain_norm_y_te, clfn.predict(pcan.transform(rain_norm_x_te))))
    
    scores.append(clf.score(pca.transform(rain_x_te_true), rain_y_te_true))
    scoresn.append(clfn.score(pcan.transform(rain_norm_x_te_true), rain_norm_y_te_true))

KeyboardInterrupt: 

In [ ]:
plt.plot(scores)

In [ ]:
plt.plot(scoresn)

In [ ]:
cmatrices = []
cmatricesn = []
scores = []
scoresn = []
for i in range(1, 30):
    pca = KernelPCA(n_components=i, kernel='rbf')
    pcan = KernelPCA(n_components=i, kernel='rbf')
    pca.fit(rain_x_tr[:5000])
    pcan.fit(rain_norm_x_tr[:5000])
    
    clf = lr(penalty='l2').fit(pca.transform(rain_x_tr), rain_y_tr)
    clfn = lr(penalty='l2').fit(pcan.transform(rain_norm_x_tr), rain_y_tr)
    
    cmatrices.append(cm(rain_y_te, clf.predict(pca.transform(rain_x_te))))
    cmatricesn.append(cm(rain_norm_y_te, clfn.predict(pcan.transform(rain_norm_x_te))))
    
    scores.append(clf.score(pca.transform(rain_x_te_true), rain_y_te_true))
    scoresn.append(clfn.score(pcan.transform(rain_norm_x_te_true), rain_norm_y_te_true))

In [ ]:
plt.plot(scores)

In [ ]:
plt.plot(scoresn)

In [ ]:
#        'Unnamed: 0', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
#        'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm',
#        'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm',
#        'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm', 'RainToday',
#        'WindGustDir_N', 'WindGustDir_S', 'WindGustDir_E', 'WindGustDir_W',
#        'WindDir9am_N', 'WindDir9am_S', 'WindDir9am_E', 'WindDir9am_W',
#        'WindDir3pm_N', 'WindDir3pm_S', 'WindDir3pm_E', 'WindDir3pm_W',
#        'RainTomorrow'

In [ ]:
manual_related = ['MinTemp', 'MaxTemp'], ['Rainfall', 'RainToday', 'Humidity9am', 'Humidity3pm']